# Example Use of Dark-MaRK Package

The Dark Mark package is designed to create bespoke predictions for the dark matter direct detectors. It offers users the ability to alter the halo model, detector, and dark matter candidate. It also allows users to calculate their differential event rate using either analytical models, or numeric integration from input simulations. 


This tutorial will demonstrate how to use Dark MaRK to test the expected signal of a dark matter direct detector, given an example simulations. *** Link to Read the Docs***

#### Import neccessary packages

In [3]:
import astropy.units as u 
import darkmark
import numpy as np 
import os

#### Import some example data - in this case we are downloading a Milky Way halo analogue from the FIRE collaborations latte suite. We download only 1/4 of the particles due to large file size.  The file is large so expect a short wait for download. The download file is then named accordingly.

In [2]:
!wget https://girder.hub.yt/api/v1/item/5b33f745323d120001bfe48d/download

--2021-10-18 16:38:13--  https://girder.hub.yt/api/v1/item/5b33f745323d120001bfe48d/download
Resolving girder.hub.yt (girder.hub.yt)... 141.142.231.2
Connecting to girder.hub.yt (girder.hub.yt)|141.142.231.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2460753960 (2.3G) [application/octet-stream]
Saving to: ‘download’

download            100%[===================>]   2.29G  4.02MB/s    in 20m 43s 

2021-10-18 16:58:57 (1.89 MB/s) - ‘download’ saved [2460753960/2460753960]



In [6]:
!mv download snapshot_600.1.hdf5

#### Load in the functions from the darkmark package that are needed

In [1]:
from darkmark.boostrap import bootstrap_rate
from darkmark.simload import load_sim_profile 
from darkmark.sampling_transformations import def_samp_coord, calc_real_uE_pE,sample_dm_solcirc 
from darkmark.dm_detector_class import Nibbler 

/Users/gracelawrence/Documents/2021/Dark-MaRK


#### Define the parameters of the experiment you want to model. Some of these parameters are not called in the example functions below.

In [10]:
v0 = 233 #Average Velocity Speed of Dark Matter in the Halo in km/s.
rho_dm = 0.3 #Dark Matter Density in (Gev/c^2)/cm^3.
M_dm = 60 #Dark Matter Particle Mass in GeV/c^2.
A = 73 #Atomic Mass of the Target Detector Nuclei.
sigma_0 = 1.3e-41*u.cm**2 #Cross-section.
v_max = 600 #Escape velocity of the system in km/s.
period = 365 #The time period to perform the calculates over (typically 1 year) in units of days.
energy_range = np.linspace(0,100,100) #The energy region of interest.
detvar =('Ge') #The detector element to investigate - Germanium.
dmvar=('CDM') #The Dark Matter cosmology assumed (CDM is currently only option with WDM and SiDM in development).
vdf_var =('SHM') #The velocity model to use, 'SHM' (standard halo model) is currntly the default.

#### Define the Nibbler class 

In [ ]:
nib = Nibbler(detvar, dmvar, 'Nucleon', vdf_var, v0,rho_dm, M_dm, sigma_0, v_max,period)                                           

#### Define the paths to the simulation data, and to the output folder

In [4]:
file_path = "./snapshot_600.1.hdf5"

In [5]:
results_path = "./DarkMark_Example_Outputs/"
CHECK_FOLDER = os.path.isdir(results_path)
if not CHECK_FOLDER:
    os.makedirs(results_path)

#### Define how many evenly spaced samples you'd like to take around the solar circle.

In [6]:
n_samples = 8 

### Load the simulation profile 

#### Load the simulation, centre and align it, and then visualize the dark matter component.

In [ ]:
target,s = load_sim_profile(True,file_path,True, True, results_path, cosmology = 'WMAP1') 

### Generate the co-ordinates at which to samples the dark matter

In [ ]:
sample_coords, theta_ang = def_samp_coord(target, n_samples) 

### Sample the dark matter and generate the velocity disitribution function in both the galactocentric and geocentric reference frames, for each of the samples.

In [ ]:
vel_galacto, vel_geo = sample_dm_solcirc(nib,sample_coords, target, theta_ang, 'milkyway',results_path, False)   

*Generate an example VDF plot comparing the galactocentric and geocentric distributions with a MB fit.*

### Integrate over the velocity distributions to generate spectral rate functions.

In [ ]:
bootstrap_rate(nib, n_samples, results_path, False) 